# Bibliotecas

In [3]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pickle
import os
import asyncio
import nest_asyncio
import random
import pandas as pd
from datetime import datetime

#para manipular google sheets
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe # para converter as planilhas em DataFrames e vice-versa
from oauth2client.service_account import ServiceAccountCredentials # para autenticar o acesso ao Google Sheets

# Criação de um template do excel para manipulação de dados
O template criado é um exemplo da estrutura de dados para armazenar membros, trilhas, cursos e suas respectivas associações

In [13]:
#tome muito cuidado para não sobrescresver o arquivo populado
NOME_TEMPLATE = 'TEMPLATE.xlsx'

membros_feadev = pd.DataFrame({
        'id_membro': [0, 1, 2],
        'nome': ['admin', 'Tiago Toledo', 'Rogério Ceni'],
        'email': ['email-falso-admin-dev@gmail.com', 'ttduarte@usp.br', 'rogerio-ceni@gmail.com'],
        'conta_github': ['admin-feadev-github', 'Tiago745', 'Rogerio-Ceni-Github'],
        'conta_datacamp': ['', '', ''], #provavelmente será uma url para cursos finalizados, ainda precisa ser revisado
        'xp_datacamp': ['', '', ''],
        'ativo': [1, 1, 0]
})

trilhas = pd.DataFrame({
        'id_trilha': [0, 1, 2], #chave/id/pk
        'nome_trilha': ['Associate Data Scientist in Python', 'Associate Data Engineer in SQL',  'Capacitação 2025 - Básico'],
        'url': ["https://app.datacamp.com/learn/career-tracks/associate-data-scientist-in-python","https://app.datacamp.com/learn/career-tracks/associate-data-engineer-in-sql", ''],
        'tipo_trilha': [0,0,1] # 0-> trilha do datacamp || 1-> trilha personalizada (default deve ser 0, 1 somente quando for personalizado ou criado in-app)
        })

cursos = pd.DataFrame({
    'id_curso': [0, 1, 2],
    'nome_curso': ['Introduction to Python', 'Intermediate Python', 'SQL'],
    'duracao' : [4, 4, 4],
    'url' : ['', '', '']
})

eventos = pd.DataFrame({
    'id_evento': [0],
    'nome_evento': ['QuantiConnect 2025 - Dia 1'],
    'tipo_evento_id': ['2'], #PK/CHAVE ESTRANGEIRA DE Tipos_de_Eventos
    'tipo_evento_nome': ['Feira'], #PK/CHAVE ESTRANGEIRA DE Tipos_de_Eventos
    'descricao': ['Primeiro dia do QuantiConnect 2025, evento organizado pela Feadev com foco em inovação quantitativa, networking e palestras.'],
    'data_inicio' : [pd.to_datetime('2025-05-10 10:00')],
    'data_fim' : [pd.to_datetime('2025-05-10 17:00')],
})

tipos_de_eventos = pd.DataFrame({
    'id_tipo_de_evento': [0, 1, 2],
    'nome_tipo_de_evento': ['Reunião', 'Apresentação', 'Feira'],
})



#DataFrames Associativos
trilhas_tem_cursos = pd.DataFrame({
    'id_trilha' : [0, 0, 1, 2, 2, 2],
    'id_curso' : [0, 1, 2, 0, 1, 2],
    'ordem_curso' : [0, 1, 0, 1, 2, 0], #ordem em que os cursos devem ser assistidos dentro de cada trilha
    'data_final_para_assistir': ['', '', '','20/06/2023', '20/06/2023', '23/06/2023'], #cursos de trilhas do datacamp nao tem data-final, apenas trilhas personalizadas
    'obrigatoriedade_curso': [1, 1, 1, 1, 1, 1] #o curso precisa ser obrigatoriamente assistido ou é opcional, 1->Obrigatório 0->Não obrigatório/opcional
})

membro_feadev_faz_trilhas = pd.DataFrame({
    'id_membro' : [1],
    'id_trilha' : [2],
    'data_inicio' : ['15/06/2025'],
    'data_fim' : ['20/06/2025'],
    'finalizado' : [True]
})

membro_feadev_faz_cursos = pd.DataFrame({
    'id_membro' : [1, 1, 1],
    'id_curso' : [0,1,2],
    'data_inicio' : ['15/06/2025', '17/06/2025', '20/06/2025'],
    'data_fim' : ['15/06/2025', '21/06/2025', '21/06/2025'],
    'finalizado' : [True, True, True]
})

membro_feadev_participa_eventos = pd.DataFrame({
    'id_membro' : [0],
    'id_evento' : [0],
    'presença' : [1], #0 = Ausente, 1 = Presente
})

#Uniao de trilhas com cursos
resultado = trilhas.merge(trilhas_tem_cursos, on='id_trilha') \
                   .merge(cursos, on='id_curso') \
                   #[['id_trilha', 'nome_trilha', 'id_curso', 'nome_curso']]  # remove colunas extras

resultado_ordenado = resultado.sort_values(by=['id_trilha', 'ordem_curso']) #ordena por trilha e depois por ordem que deve assistir cada curso, facilita a leitura

print(resultado_ordenado[['id_trilha', 'nome_trilha', 'id_curso', 'nome_curso', 'ordem_curso']])

#limpando as linhas de lorem ipsum em trilhas e cursos
trilhas = trilhas[0:0]
cursos = cursos[0:0]

# Caminho do arquivo Excel que será salvo
arquivo_excel = 'dados_trilhas.xlsx'

# Usando o ExcelWriter para salvar cada DataFrame em uma aba diferente
with pd.ExcelWriter(NOME_TEMPLATE, engine='openpyxl') as writer:
    trilhas.to_excel(writer, sheet_name='trilhas', index=False)
    cursos.to_excel(writer, sheet_name='cursos', index=False)
    eventos.to_excel(writer, sheet_name='eventos', index=False)
    tipos_de_eventos.to_excel(writer, sheet_name='tipos_de_eventos', index=False)
    trilhas_tem_cursos.to_excel(writer, sheet_name='trilhas_tem_cursos', index=False)
    membros_feadev.to_excel(writer, sheet_name='membros_feadev', index=False)
    membro_feadev_faz_trilhas.to_excel(writer, sheet_name='membro_feadev_faz_trilhas', index=False)
    membro_feadev_faz_cursos.to_excel(writer, sheet_name='membro_feadev_faz_cursos', index=False)
    membro_feadev_participa_eventos.to_excel(writer, sheet_name='membro_feadev_participa_eventos', index=False)

print("✅ Arquivo Excel salvo com sucesso!")


   id_trilha                         nome_trilha  id_curso  \
0          0  Associate Data Scientist in Python         0   
1          0  Associate Data Scientist in Python         1   
2          1      Associate Data Engineer in SQL         2   
5          2           Capacitação 2025 - Básico         2   
3          2           Capacitação 2025 - Básico         0   
4          2           Capacitação 2025 - Básico         1   

               nome_curso  ordem_curso  
0  Introduction to Python            0  
1     Intermediate Python            1  
2                     SQL            0  
5                     SQL            0  
3  Introduction to Python            1  
4     Intermediate Python            2  
✅ Arquivo Excel salvo com sucesso!


# Funções para manipular os dados
- Atualizar trilhas e cursos do datacamp, associação entre cursos e trilhas
- Registro de presença em eventos da Feadev
- Registra interações entre membros da Feadev e cursos ou trilhas
- Funções de listagem

In [5]:
import utils as ut

# Variáveis Estáticas e autenticação JSON
### Algumas funções usarão implicitamente estas variáveis, então é preciso defini-las antes de rodar o código
O github não permite o envio de arquivos sensíveis como JSON com dados de autenticação, entao armazene localmente o arquivo

In [ ]:
#Google Sheets API
#Google Drive API
ID_DA_PLANILHA_GOOGLE_SHEETS = '1TneLohY9QvCDMCJAS0fCdNXsqxEnVw-YsRl3DpzVDdY' #fica entre /d/ e /edit na URL.

#ARQUIVO DE PERTENCIMENTO DA FEADEV
LOCAL_CREDENCIAIS_JSON_DE_ACESSO = '../credenciais.json' #Coloque o endereço do arquivo no seu PC


# Escopos de acesso
ESCOPOS = [
    "https://spreadsheets.google.com/feeds", # acesso ao Google Sheets
    "https://www.googleapis.com/auth/spreadsheets", # acesso ao Google Sheets
    "https://www.googleapis.com/auth/drive" # acesso ao Google Drive
]




# Exemplo de como chamar as funções

In [8]:
key_google = ID_DA_PLANILHA_GOOGLE_SHEETS
cliente = ut.abrir_cliente_JSON()
id_do_membro=1

ut.listar_trilhas_feitas_por_um_membro(key_google, cliente, id_do_membro)

,id_trilha,nome_trilha,url,tipo_trilha,data_inicio,data_fim,finalizado
0,2,Capacitação Avançada 2025,https://app.datacamp.com/learn/career-tracks/c...,1.0,2025-06-01 00:00:00,2025-06-15 00:00:00,1.0
1,1,Associate Data Scientist in Python,https://app.datacamp.com/learn/career-tracks/a...,0.0,2025-06-01 00:00:00,2025-06-15 00:00:00,1.0


# Atualizar cursos e trilhas do datacamp
Inicalmente atualiza as sheets de [Trilhas] e de [Cursos], depois disso faz a associação entre os ids de trilhas e cursos, gerando a sheet [Trilhas_tem_Cursos]

In [ ]:
# Autenticação usando o JSON baixado
credenciais = ServiceAccountCredentials.from_json_keyfile_name(LOCAL_CREDENCIAIS_JSON_DE_ACESSO, ESCOPOS) #Credenciais do JSON

cliente = gspread.authorize(credenciais)

# Abre a planilha
planilha = cliente.open_by_key(ID_DA_PLANILHA_GOOGLE_SHEETS)

# Lê todas as abas em um dicionário: {nome_aba: dataframe}
planilhas = {
    aba.title: get_as_dataframe(aba).dropna(how="all")
    for aba in planilha.worksheets() #Esse comando retorna uma lista de objetos aba, ou seja, todas as guias/abas da planilha do Google Sheets.
}

#Acessa as abas especificas Trilhas e Cursos
df_sheet_trilhas_maedev = planilhas['trilhas']
df_sheet_cursos_maedev = planilhas['cursos']

#Atualiza baseado em arquivo local
for trilha in ut.LISTA_DE_TRILHAS_PARA_LER:
    trilhas_atualizadas = pd.read_excel(f'./{ut.PASTA_TRILHAS}/{trilha}') #Essa lista de trilhas para ler é arquivo local (vem do scrapping do datacamp)
    df_sheet_trilhas_maedev = ut.atualizar_trilhas_com_ids(trilhas_atualizadas, df_sheet_trilhas_maedev) #Atualiza o DF de trilhas
    df_sheet_cursos_maedev = ut.atualizar_cursos_com_ids(trilhas_atualizadas, df_sheet_cursos_maedev) #Atualiza o DF de cursos



# Sobrescrevendo o arquivo original (unindo de volta ao DF com todas as abas)
planilhas['trilhas'] = df_sheet_trilhas_maedev
planilhas['cursos'] = df_sheet_cursos_maedev



# Salva todas as abas de volta (sobrescrevendo o arquivo original)
for nome_sheet, df in planilhas.items(): #Para cada aba
    try:
        # Tenta abrir a aba existente
        aba = planilha.worksheet(nome_sheet)
        # Limpa o conteúdo antigo
        aba.clear()
    except gspread.WorksheetNotFound:
        # Se a aba não existir, cria nova
        aba = planilha.add_worksheet(title=nome_sheet, rows=str(len(df)+10), cols=str(len(df.columns)+10))

    # Escreve o DataFrame atualizado na aba
    set_with_dataframe(aba, df)


# Associa as trilhas com os cursos
ut.gerar_trilhas_tem_cursos(cliente, ID_DA_PLANILHA_GOOGLE_SHEETS, ut.PASTA_TRILHAS)


# Criando cookies 
Antes de puxar as trilhas do datacamp, é necessário logar e gerar os cookies

In [ ]:
from playwright.async_api import async_playwright # Rode 'playwright install' no terminal para instalar os navegadores quando for usar pela primeira vez
import pickle
import asyncio
import os
import nest_asyncio # Se estiver rodando no Jupyter ou Colab, use 'nest_asyncio' para permitir chamadas assíncronas

nest_asyncio.apply()  # permite rodar async no Jupyter/Colab

url = "https://app.datacamp.com"
cookies_path = "./cookies/datacamp_cookies.pkl"

async def save_cookies_manually():
    os.makedirs(os.path.dirname(cookies_path), exist_ok=True)

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        await page.goto(url, timeout=60000)
        await context.clear_cookies()

        input("⚠️ Faça login no navegador que abriu. Depois pressione Enter aqui...")

        cookies = await context.cookies()
        with open(cookies_path, "wb") as f:
            pickle.dump(cookies, f)

        print(f"✅ Cookies salvos com sucesso em: {cookies_path}")

# Em Jupyter ou Colab:
await save_cookies_manually()

✅ Cookies salvos com sucesso em: ./cookies/datacamp_cookies.pkl


# Gerando arquivo com os cursos do datacamp
### Gera um xlsx com a ata de cursos de cada trilha dentro da lista LISTA_DE_TRILHAS_PARA_ATUALIZAR

In [ ]:


'''
Mude as urls para atualizar trilhas diferentes do datacamp, 
Para ser mais rápido, atualize uma por vez e eviter atualizar coisas desnecessárias
Funciona tanto para skill tracks quanto carrer tracks
'''
LISTA_DE_TRILHAS_PARA_ATUALIZAR = [
    #trilhas da capacitação básica 2025
    #"https://app.datacamp.com/learn/career-tracks/data-scientist-with-python",
    #'https://app.datacamp.com/learn/skill-tracks/sql-fundamentals',
    #'https://app.datacamp.com/learn/skill-tracks/git-fundamentals',
    #'https://app.datacamp.com/learn/skill-tracks/excel-fundamentals',

    #trilhas da capacitação avançada 2025
    #'https://app.datacamp.com/learn/skill-tracks/time-series-with-python', #curso de Séries temporais
    #'https://app.datacamp.com/learn/skill-tracks/applied-finance-in-python', #curso de Introdução ao Gerenciamento de Risco de Portfólio em Python e curso de Garch
    #'https://app.datacamp.com/learn/skill-tracks/finance-fundamentals-in-python', #curso de Introdução à Análise de Portfólio em python
    'https://app.datacamp.com/learn/career-tracks/machine-learning-scientist-with-python' #contem boa parte dos cursos de machine learning usados
     
]
'''
CURSOS SEM TRILHAS (PRECISAM SER ADICIONADOS MANUALMENTE)
https://app.datacamp.com/learn/courses/introduction-to-optimization-in-python
https://app.datacamp.com/learn/courses/introduction-to-deep-learning-in-python
'''

nest_asyncio.apply()

cookies_path = "./cookies/datacamp_cookies.pkl"
semaphore = asyncio.Semaphore(3)

# Cria o contexto autenticado com os cookies
async def create_authenticated_context(playwright):
    if not os.path.exists(cookies_path):
        print("❌ Arquivo de cookies não encontrado. Faça o login e salve os cookies primeiro.")
        return None, None

    browser = await playwright.chromium.launch(headless=False)
    context = await browser.new_context()

    with open(cookies_path, "rb") as f:
        cookies = pickle.load(f)
    await context.add_cookies(cookies)

    return browser, context

# Extrai os cursos e durações de uma trilha
async def extract_courses(context, url):
    async with semaphore:
        page = await context.new_page()
        try:
            await page.goto(url, timeout=60000)
            await asyncio.sleep(random.uniform(2, 4))

            if "just a moment" not in (await page.content()).lower():
                soup = BeautifulSoup(await page.content(), "html.parser")

                track_element = soup.find("h1")
                track = track_element.get_text(strip=True) if track_element else "N/A"

                projects = {
                    optional.find_parent("div")
                    for optional in soup.select(".mfe-app-learn-hub-1moscjt")
                }

                courses = list([
                    course.get_text(strip=True)
                    for course in soup.select("h3.mfe-app-learn-hub-1yqo1j7")
                    if course.find_parent("div") not in projects
                ])
                await page.close()

                duration = {}
                for course in courses:
                    course_page = course.replace(" ", "-").lower()
                    tmp_url = f"https://app.datacamp.com/learn/courses/{course_page}"
                    page = await context.new_page()
                    try:
                        await page.goto(tmp_url, timeout=60000)
                        await asyncio.sleep(random.uniform(2, 3))

                        soup = BeautifulSoup(await page.content(), "html.parser")
                        duration_element = soup.select_one(".mfe-app-learn-hub-hdd90k")
                        duration[course] = duration_element.get_text(strip=True) if duration_element else "N/A"
                    except Exception as e:
                        print(f"⚠️ Erro ao acessar curso '{course}': {e}")
                        duration[course] = "N/A"
                    finally:
                        await page.close()

                data = [{"Trilha": track, "Curso": course, "Duração": duration[course]} for course in courses]
                print(f"✅ Cursos extraídos com sucesso da trilha: {track}")
                return data
            else:
                print("❌ Cookies expirados ou inválidos. Refaça o login.")
                return None
        except Exception as e:
            print(f"❌ Erro ao extrair curso da página {url}: {e}")
            return None

# Função principal
async def main(url_da_trilha):
    async with async_playwright() as p:
        browser, context = await create_authenticated_context(p)
        if context is None:
            return

        data = await extract_courses(context, url_da_trilha)

        await browser.close()

        if data:
            df = pd.DataFrame(data)
            df['ordem_para_assistir'] = range(len(df)) #ordem de acordo com o html da página
            df['tipo_trilha'] = 0 #0->Trilha do datacamp, 1->trilha personalizada
            df['data_final_para_assistir'] = '' #cursos do datacamp não tem datas especificas, isso só se aplica para trilhas personalizadas
            df['obrigatoriedade_curso'] = 1 #cursos do datacamp por default sao obrigatorios, opcionalidade só se aplica para trilhas personalizadas
            print(df)
            return df
        else:
            print("Nenhum dado extraído.")
            return None


# Loop principal para pegar todas as trilhas da lista
async def rodar_todas_as_trilhas():
    for trilha_do_datacamp in LISTA_DE_TRILHAS_PARA_ATUALIZAR:
        df = await main(trilha_do_datacamp)
        if df is not None:
            df.to_excel((f'trilhas_atualizadas/{df.iloc[0]["Trilha"]}.xlsx'), index=False) #Salva o arquivo com titulo da trilha

#Executa a função
await rodar_todas_as_trilhas()

✅ Cursos extraídos com sucesso da trilha: Machine Learning Scientist in Python
                                  Trilha  \
0   Machine Learning Scientist in Python   
1   Machine Learning Scientist in Python   
2   Machine Learning Scientist in Python   
3   Machine Learning Scientist in Python   
4   Machine Learning Scientist in Python   
5   Machine Learning Scientist in Python   
6   Machine Learning Scientist in Python   
7   Machine Learning Scientist in Python   
8   Machine Learning Scientist in Python   
9   Machine Learning Scientist in Python   
10  Machine Learning Scientist in Python   
11  Machine Learning Scientist in Python   
12  Machine Learning Scientist in Python   
13  Machine Learning Scientist in Python   
14  Machine Learning Scientist in Python   
15  Machine Learning Scientist in Python   
16  Machine Learning Scientist in Python   
17  Machine Learning Scientist in Python   
18  Machine Learning Scientist in Python   
19  Machine Learning Scientist in Python 